In [1]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '~/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 256

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    dataX=pd.read_csv(datasetFolder + datasetXFile, engine='c', na_filter=False)
    dataY=pd.read_csv(datasetFolder + datasetYFile, engine='c', na_filter=False)

    # delete the id fields
    del dataX['id']
    del dataY['id']

    return dataX, dataY

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

Loading datasetX_1.csv & datasetY_1.csv


In [5]:
dataX.head()

,walk_1,walk_2,walk_3,walk_4,walk_5,walk_6,walk_7,walk_8,walk_9,walk_10,...,walk_1203,walk_1204,walk_1205,walk_1206,walk_1207,walk_1208,walk_1209,walk_1210,walk_1211,walk_1212
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dataY.head()

,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,...,class_530,class_531,class_532,class_533,class_534,class_535,class_536,class_537,class_538,class_539
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 1212
Output Classes: 539


In [8]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu'))
deepModel.add(Dense(1024, activation='relu'))
deepModel.add(Dense(768, activation='relu'))
deepModel.add(Dense(512, activation='relu'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid'))

models = [logisticRegression, simpleModel, deepModel]

In [21]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [22]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX.as_matrix(), dataY.as_matrix(), nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX.as_matrix(), dataY.as_matrix())

In [23]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [24]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for eon in range(0, numberOfEons):
    print('\n***********************************\nEon {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        print('Training...')
        for model in models:
            #print('Model = {}'.format(model.name))
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[trainIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX.as_matrix())
            true_positives, predicted_positives, possible_positives = countPredictions(dataY.as_matrix(), predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}. Eon {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
Eon 1/5
Learning for file 1 / 192 : datasetX/Y_190
Loading datasetX_190.csv & datasetY_190.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.5606 	 f1-score = 0.3575
   Model = 2 Fully Connected Layers 	 loss = 0.0228 	 f1-score = 0.0003
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0393 	 f1-score = 0.1216
Learning for file 2 / 192 : datasetX/Y_185
Loading datasetX_185.csv & datasetY_185.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.5598 	 f1-score = 0.3548
   Model = 2 Fully Connected Layers 	 loss = 0.0222 	 f1-score = 0.0137
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0344 	 f1-score = 0.1744
Learning for file 3 / 192 : datasetX/Y_235
Loading datasetX_235.csv & datasetY_235.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.5589 	 f1-score = 0.3613
   Model = 2 Fully Connected Layers 	 loss = 0.0220 	 f1-score = 0.0138
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0353 	 f1-score = 0.

==================================================
# Relationship Presence
==================================================
1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.3838	 precision = 0.8991 	 recall = 0.2440
 - Model = 2 Fully Connected Layers 	 f1-score = 0.5441	 precision = 0.8756 	 recall = 0.3948
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.5750	 precision = 0.8791 	 recall = 0.4273
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.4146	 precision = 0.8938 	 recall = 0.2699
 - Model = 2 Fully Connected Layers 	 f1-score = 0.5691	 precision = 0.8745 	 recall = 0.4218
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.5834	 precision = 0.8839 	 recall = 0.4354
3. Eon 3/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.4409	 precision = 0.8876 	 recall = 0.2933
 - Model = 2 Fully Connected Layers 	 f1-score = 0.5757	 precision = 0.8790 	 recall = 0.4280
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.5869	 precision = 0.8781 	 recall = 0.4408
4. Eon 4/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.4579	 precision = 0.8801 	 recall = 0.3095
 - Model = 2 Fully Connected Layers 	 f1-score = 0.5806	 precision = 0.8794 	 recall = 0.4334
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.5880	 precision = 0.8781 	 recall = 0.4420
5. Eon 5/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.4745	 precision = 0.8773 	 recall = 0.3253
 - Model = 2 Fully Connected Layers 	 f1-score = 0.5832	 precision = 0.8789 	 recall = 0.4364
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.5868	 precision = 0.8833 	 recall = 0.4394